In [1]:
# --------------------------------------------------------------------------
# ۱. نصب کتابخانه‌های مورد نیاز
# wandl: e3b067db7cb39378cff354e9738c4095f9dbb0d5
# --------------------------------------------------------------------------
print("مرحله ۱: در حال نصب کتابخانه‌های مورد نیاز...")
!pip install -q -U transformers datasets accelerate peft bitsandbytes trl torch
print("نصب کتابخانه‌ها با موفقیت انجام شد.")
print("-" * 50)


# --------------------------------------------------------------------------
# ۲. وارد کردن کتابخانه‌ها
# --------------------------------------------------------------------------
print("مرحله ۲: در حال وارد کردن کتابخانه‌ها...")
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
import re
from collections import Counter
import string
print("وارد کردن کتابخانه‌ها با موفقیت انجام شد.")
print("-" * 50)


# --------------------------------------------------------------------------
# ۳. بارگذاری و آماده‌سازی مجموعه داده PersianQA
# --------------------------------------------------------------------------
print("مرحله ۳: در حال بارگذاری و آماده‌سازی مجموعه داده...")
dataset_name = "SajjadAyoubi/persian_qa"
dataset = load_dataset(dataset_name)

def create_prompt(example):
    """ایجاد یک قالب استاندارد برای ورودی مدل."""
    if example['answers'] and example['answers']['text']:
        prompt = f"""### زمینه:
{example['context']}

### پرسش:
{example['question']}

### پاسخ:
{example['answers']['text'][0]}"""
        return {"text": prompt}
    return {"text": None}

train_dataset = dataset['train'].map(create_prompt).filter(lambda x: x['text'] is not None)
eval_dataset = dataset['validation'].map(create_prompt).filter(lambda x: x['text'] is not None)
print("مجموعه داده با موفقیت آماده شد.")
print("نمونه پرامپت ساخته شده:")
print(train_dataset[0]['text'])
print("-" * 50)


# --------------------------------------------------------------------------
# ۴. بارگذاری مدل و توکنایزر با کوانتیزیشن ۴-بیت (QLoRA)
# --------------------------------------------------------------------------
print("مرحله ۴: در حال بارگذاری مدل و توکنایزر...")
model_name = "unsloth/llama-3.2-1b-bnb-4bit"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
print("مدل و توکنایزر با موفقیت بارگذاری شدند.")
print("-" * 50)


# --------------------------------------------------------------------------
# ۵. تنظیمات LoRA
# --------------------------------------------------------------------------
print("مرحله ۵: در حال اعمال تنظیمات LoRA...")
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)

model = get_peft_model(model, lora_config)
print("تنظیمات LoRA با موفقیت اعمال شد.")
print("-" * 50)


# --------------------------------------------------------------------------
# ۶. شروع فرآیند فاین-تیون (Fine-Tuning)
# --------------------------------------------------------------------------
print("مرحله ۶: در حال شروع فرآیند فاین-تیون...")

# تعریف پارامترهای آموزش
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    max_steps=70,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=20,
    save_total_limit=2,
)

# ساخت ترینر
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=lora_config,
    args=training_args,
)

# شروع آموزش
trainer.train()
print("فرآیند فاین-تیون با موفقیت به پایان رسید.")
print("-" * 50)


# --------------------------------------------------------------------------
# ۷. ارزیابی مدل و گزارش نتایج
# --------------------------------------------------------------------------
print("مرحله ۷: در حال ارزیابی مدل و محاسبه امتیازات...")

def normalize_text(s):
    """حذف علائم نگارشی و فاصله‌های اضافی از متن"""
    s = "".join(c for c in s if c not in string.punctuation)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

def compute_f1(prediction, ground_truth):
    """محاسبه F1-Score"""
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(ground_truth).split()
    common = Counter(pred_tokens) & Counter(truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_tokens) if len(pred_tokens) > 0 else 0
    recall = 1.0 * num_same / len(truth_tokens) if len(truth_tokens) > 0 else 0
    if precision + recall == 0:
        return 0
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def compute_em(prediction, ground_truth):
    """محاسبه Exact Match"""
    return 1 if normalize_text(prediction) == normalize_text(ground_truth) else 0

eval_samples = dataset['validation'].select(range(100))
predictions = []
ground_truths = []

model.eval()

for sample in eval_samples:
    prompt = f"""### زمینه:
{sample['context']}

### پرسش:
{sample['question']}

### پاسخ:
"""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=64, pad_token_id=tokenizer.eos_token_id)

    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    try:
        answer = full_output.split("### پاسخ:")[1].strip()
    except IndexError:
        answer = ""

    if sample['answers'] and sample['answers']['text']:
        predictions.append(answer)
        ground_truths.append(sample['answers']['text'][0])

total_f1 = 0
total_em = 0
for pred, truth in zip(predictions, ground_truths):
    total_f1 += compute_f1(pred, truth)
    total_em += compute_em(pred, truth)

final_f1_score = (total_f1 / len(predictions)) * 100 if predictions else 0
final_em_score = (total_em / len(predictions)) * 100 if predictions else 0

print("ارزیابی با موفقیت انجام شد.")
print("-" * 50)


# --------------------------------------------------------------------------
# نمایش نتایج نهایی
# --------------------------------------------------------------------------
print("\n--- نتایج ارزیابی نهایی ---")
print(f"Exact Match (EM): {final_em_score:.2f}%")
print(f"F1-Score: {final_f1_score:.2f}%")
print("--------------------------\n")

# نمایش چند نمونه از پاسخ‌های تولید شده
print("نمونه پاسخ‌های تولید شده توسط مدل:")
for i in range(min(5, len(predictions))):
    print(f"\nپرسش: {eval_samples[i]['question']}")
    print(f"پاسخ صحیح: {ground_truths[i]}")
    print(f"پاسخ مدل: {predictions[i]}")

مرحله ۱: در حال نصب کتابخانه‌های مورد نیاز...
نصب کتابخانه‌ها با موفقیت انجام شد.
--------------------------------------------------
مرحله ۲: در حال وارد کردن کتابخانه‌ها...
وارد کردن کتابخانه‌ها با موفقیت انجام شد.
--------------------------------------------------
مرحله ۳: در حال بارگذاری و آماده‌سازی مجموعه داده...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using the latest cached version of the dataset since SajjadAyoubi/persian_qa couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'persian_qa' at /root/.cache/huggingface/datasets/SajjadAyoubi___persian_qa/persian_qa/1.0.0/5a314e73c690b159983b2e45b9d4c0500a80cfd0 (last modified on Fri Sep 12 20:01:42 2025).


Map:   0%|          | 0/9008 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9008 [00:00<?, ? examples/s]

Map:   0%|          | 0/930 [00:00<?, ? examples/s]

...

Truncating train dataset:   0%|          | 0/6306 [00:00<?, ? examples/s]

wandb: Currently logged in as: s-hnj1381 (s-hnj1381-university-of-guilan) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
20,2.498500
40,2.414300
60,2.359600


فرآیند فاین-تیون با موفقیت به پایان رسید.
--------------------------------------------------
مرحله ۷: در حال ارزیابی مدل و محاسبه امتیازات...
ارزیابی با موفقیت انجام شد.
--------------------------------------------------

--- نتایج ارزیابی نهایی ---
Exact Match (EM): 18.57%
F1-Score: 42.81%
--------------------------

نمونه پاسخ‌های تولید شده توسط مدل:

پرسش: پایتخت اسپانیا کجاست؟
پاسخ صحیح: مادرید
پاسخ مدل: مادرید

پرسش: بر چه اساسی رئال موفق ترین تیم در تاریخ فوتبال اروپا است؟
پاسخ صحیح: فیفا
پاسخ مدل: در لالیگا، از تیم‌های قدرتمند محسوب می‌شود

پرسش: رئال مادرید چند بار در لیگ قهرمانان اروپا به عنوان قهرمانی رسیده؟
پاسخ صحیح: ۱۳
پاسخ مدل: ۱۳ قهرمانی

پرسش: معنی واژه رئال به اسپانیایی چیست؟
پاسخ صحیح: سلطنتی
پاسخ مدل: سلطنتی

پرسش: تیم رئال مادرید برای کجاست؟
پاسخ صحیح: مادرید، پایتخت اسپانیا
پاسخ مدل: پایتخت اسپانیا
